In [1]:
import json
import requests as rq
import pandas as pd
import time

apikey = '9c594b29-ad69-4833-b88b-db5cdfa30683'

import pymongo
from pymongo import MongoClient
client = MongoClient()
db = client.lol_match_db

In [2]:
list_matches = pd.read_excel('MatchIDList_NoDupes.xlsx',
                      sheetname = 'match_id_list',
                      header = 0,
                      index_col = 0,
                      parse_cols = "B",
                      convert_float = True)
list_matches.head()

""
match_id
2853586095
2807676541
2807350386
2804711583
2795203055


In [6]:
print list_matches.index[0]
print len(list_matches)

2853586095
225128


In [11]:
#first try
r = rq.get('https://euw.api.pvp.net/api/lol/euw/v2.2/match/'
           +str(list_matches.index[0])+'?includeTimeline=TRUE&api_key=' + apikey)
print r.status_code

200


In [14]:
result = db.match_details_2016.insert_one({"_id": str(list_matches.index[0]), "match_detail":json.loads(r.text)})

print 'Inserted at:' + result.inserted_id

Inserted at:2853586095


In [31]:
#step by step, let's start by doing 1000
#second step, up to 10.000
#third step, up to 20.000
#third step, up to 35.000 while the other thread goes up to 50.000
#up to 150.000 now
#up to 200.000

i=212500
while i < 225128:
    #get all matches for a summoner id
    #WARNING: let's do only one request every two seconds
    r = rq.get('https://euw.api.pvp.net/api/lol/euw/v2.2/match/' +str(list_matches.index[i])
               +'?includeTimeline=TRUE&api_key=' + apikey)
    #check response code
    while (r.status_code <> 200):
        time.sleep(1.5)
        r = rq.get('https://euw.api.pvp.net/api/lol/euw/v2.2/match/' +str(list_matches.index[i])
               +'?includeTimeline=TRUE&api_key=' + apikey)
    
    result = db.match_details_2016.insert_one({"_id": str(list_matches.index[i]), "match_detail":json.loads(r.text)})
    #insert into match_list_2016(contains all matches for Season 2016 for any particular summoner) using
    #the summoner ID as DB Key    
    #increase and wait    
    i = i + 1
    time.sleep(1.25)

In [32]:
print i

225128
